In [2]:
import pandas as pd
import spacy
import en_core_web_sm
import random

In [33]:
### Procédure de nettoyage ###
data_raw = pd.read_csv('pictures_sample.csv')
data_english = data_raw[data_raw.Language == 'EN']
def author_del(text):
    text_s = text
    while(text_s[-1] != '\n' and len(text_s) > 1): 
        text_s = text_s[:-1]
    if(len(text_s) <= 1):
        return(text)
    return(text_s[:-1])
data_english.Description = data_english.Description.apply(author_del)          
data_english.drop_duplicates(subset='Description', keep='first', inplace=True)
ind = list(data_english.index)
##############################

c:\users\dimit\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (3,21,22,33,35,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\dimit\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
c:\users\dimit\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 

In [54]:
### Procédure de lecture ###
ent_load = pd.read_pickle("ent.csv")
M = ent_load.to_numpy()
M[M == None] = 0
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] == 0:
            M[i,j] = []
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i,j] != []:
            M[i,j] = [M[i,j]]
M = list(M.sum(axis = 1))
def freq(L):
    List = [l[0][0] for l in L]
    List_unique = list(set(List))
    freqs = []
    for l in List_unique:
        f = 0
        for ll in List:
            if l == ll:
                f+=1
        f /= len(List)
        freqs.append(f)
    freqs = [(List_unique[i], freqs[i]) for i in range(len(List_unique))]
    return(dict(freqs))
Freq = [freq(m) for m in M]
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}
for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]
def dep_to_val(l):
    L = []
    for i in range(len(l)):
        if l[i] in dict_val.keys():
            L.append(dict_val[l[i]])
        else:
            L.append(0)
    return(max(L))
def final(l):
    L = [] 
    for i in l:
        ll = [] 
        for ii in i:
            ll.append((ii[0], dep_to_val(ii[1])))
        L.append(ll)
    for i in range(len(L)):
        name = L[i][0][0]
        score = 0
        for ii in L[i]:
            score += ii[1]
        L[i] = (name, score)
    return(L)
def clean_final(l):
    L = []
    for i in l:
        ll = []
        for ii in i:
            if ii not in ll:
                ll.append(ii)
        L.append(ll)
    return(L)
M = [final(m) for m in M]
M = clean_final(M)
M = [dict(m) for m in M]
############################

In [69]:
# On définit le score entre deux descriptions comme la somme des moyennes des scores pour chaque entité contenue dans les deux
# textes en même temps

def similarity_text(s1, s2, a, b):
    score = 0
    for i in s1.keys():
        if i in s2.keys():
            score += (s1[i]+s2[i])/2 - abs(Freq[a][i]-Freq[b][i])/2
    return(score)    

In [73]:
# On crée une fonction qui renvoit les indices des n plus grands éléments de la liste

def n_best(l, n):
    return([j[0] for j in sorted([(i,l[i]) for i in range(len(l))],key=lambda tup: tup[1], reverse = True)[0:n+1]])

# On affiche le score pour 100 descriptions aléatoires où le score est définit comme 1 si la description associée est la même
# et 0 sinon

r = []
R = []
D = []

n = 100

for _i in range(n):
    num = random.randint(0, data_english.shape[0])
    while(num in r):
        num = random.randint(0, data_english.shape[0])
    r.append(num)
    
somme = 0

for i in range(n):
    Scores = []
    for j in range(data_english.shape[0]):
        Scores.append(similarity_text(M[r[i]], M[j], r[i], j))
    Scores_argmax = n_best(Scores, 100)
    if r[i] in Scores_argmax:
        somme += 1
    else:
        R.append(r[i])
        D.append(Scores_argmax)
        
score_moyen = somme/n

print(score_moyen)

0.98


In [74]:
print(data_english.Description[ind[R[0]]])
print('\n')
print('\n')
for i in range(10):
    print(data_english.Description[ind[D[0][i]]])
    print('\n')

A metro driver take the start at an Athens' metro station on January 11, 2013.      AFP PHOTO / ARIS MESSINIS




Protesters stand near the Athens court, during a demonstration in Athens on January 12, 2013. Several thousand people demonstrated in Athens to protest recent police operations against squatters in public buildings that have been condemned by the opposition. The protesters -- estimated by police at around 3,500 and nearly double that by media -- marched to the Athens court complex and police headquarters to demand the release of scores of individuals arrested in the sweeps. One of the central Athens buildings, a 19th-century former school, had been occupied for over two decades. The other is a stately block of flats whose former occupants include legendary soprano Maria Callas before her rise to fame. AFP PHOTO/ LOUISA GOULIAMAKI


Protesters clash with the riot police during a demonstration in Athens on January 12, 2013. Several thousand people demonstrated in Athens to pr